In [3]:
import numpy as np
import pandas as pd
import os
import csv
import ast
import seaborn as sns
from matplotlib import pyplot as plt
import wquantiles 

from tqdm import tqdm
from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.examples.utils_example import (
    dataframe_by_group,
    df_weighted_average_grouped,
    wavg)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import new_get_inflators_by_year, new_get_cn_aggregates
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.Utils import weighted_quantiles
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [1]:
simulated_variables = ['aise',
'depenses_totales',
'depenses_tot',
'rev_disponible',
 'niveau_de_vie',
 'niveau_vie_decile',
 'decile_indiv_niveau_vie',
 'ocde10',
 'pondmen',
 'nactifs',
 'npers',
 'identifiant_menage']

In [41]:
# Première simulation : données BdF 2017 brutes
year = 2017
data_year = 2017
tax_benefit_system = FranceIndirectTaxationTaxBenefitSystem()

survey_scenario = SurveyScenario.create(
    tax_benefit_system = tax_benefit_system,
    year = year,
    data_year = data_year
    )

In [40]:
def epargne(df):
    df['part_epargnants'] = (df['rev_disponible'] >= df['depenses_tot']).astype(float)
    df['aise_1_2_3'] = (df['aise'] <= 3).astype(float)
    df_by_decile = df_weighted_average_grouped(dataframe = df, groupe = 'decile_indiv_niveau_vie', varlist = ['depenses_tot','rev_disponible','niveau_de_vie','part_epargnants','aise_1_2_3'])
    df_by_decile['taux_epargne'] = 1 - (df_by_decile ['depenses_tot'] / df_by_decile ['rev_disponible'])

    return(df_by_decile)
    

In [42]:
bdf_brut_2017 = survey_scenario.create_data_frame_by_entity(simulated_variables, period = 2017)['menage']
bdf_brut_2017_by_decile = epargne(bdf_brut_2017)

In [43]:
bdf_brut_2017_by_decile

,depenses_tot,rev_disponible,niveau_de_vie,part_epargnants,aise_1_2_3,taux_epargne
decile_indiv_niveau_vie,,,,,,
1.0,16624.379693,12033.365675,7771.597011,0.364042,0.556694,-0.381524
2.0,18672.769979,18695.775843,12194.837137,0.606430,0.646267,0.001231
3.0,20159.614604,22062.551965,14711.467432,0.666473,0.717900,0.086252
4.0,22786.992379,25747.384567,17021.682150,0.696474,0.800655,0.114978
5.0,24192.691589,28938.312244,19152.627451,0.723806,0.840285,0.163991
6.0,27054.696501,32627.493487,21407.492165,0.748634,0.875082,0.170801
7.0,29309.233033,37174.906606,23894.011292,0.782197,0.917643,0.211586
8.0,32712.273305,42383.481753,27136.563509,0.791964,0.927321,0.228183
9.0,35989.341316,50143.268038,32092.461224,0.855351,0.948395,0.282270


In [22]:
# Deuxième simulation : données BdF calées sur la compta nat en 2017 
inflators_by_year = new_get_inflators_by_year(rebuild = False, year_range = range(2017, 2025), data_year = data_year)
inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])

survey_scenario = SurveyScenario.create(
    inflation_kwargs =  inflation_kwargs,
    tax_benefit_system = tax_benefit_system,
    year = year,
    data_year = data_year
    )

In [23]:
bdf_cale_2017 = survey_scenario.create_data_frame_by_entity(simulated_variables, period = 2017)['menage']
bdf_cale_2017_by_decile = epargne(bdf_cale_2017)

In [24]:
bdf_cale_2017_by_decile

,depenses_tot,rev_disponible,niveau_de_vie,part_epargnants,aise_1_2_3,taux_epargne
decile_indiv_niveau_vie,,,,,,
1.0,20883.408955,14621.500769,9443.111331,0.378410,0.556694,-0.428267
2.0,23188.843481,22716.861486,14817.701543,0.633176,0.646267,-0.020777
3.0,25013.418878,26807.763471,17875.608407,0.674252,0.717900,0.066934
4.0,28424.274452,31285.129565,20682.703918,0.698420,0.800655,0.091445
5.0,31097.516979,35162.361638,23271.972713,0.721730,0.840285,0.115602
6.0,35293.039565,39645.011599,26011.813482,0.744380,0.875082,0.109774
7.0,36002.262567,45170.481892,29033.132894,0.775009,0.917643,0.202969
8.0,41812.952536,51499.316872,32973.092990,0.774569,0.927321,0.188087
9.0,44315.099879,60928.077225,38994.904632,0.842329,0.948395,0.272665


In [33]:
# Comparaison avec l'ERFS 
erfs_path = "C:/Users/veve1/OneDrive/Documents/ENSAE 3A/Memoire MiE/Data/erfs_fpr/2017/csv"
erfs_menage_2017 = pd.read_csv(os.path.join(erfs_path,"fpr_menage_2017.csv"), sep = ";")
erfs_mrf17 = pd.read_csv(os.path.join(erfs_path,"fpr_mrf17e17t4.csv"), sep = ";") 

erfs_2017 = erfs_menage_2017.merge(erfs_mrf17, how = "left", left_on='ident17', right_on = 'ident17')
erfs_2017.columns = erfs_2017.columns.str.lower()
erfs_2017.rename({'wprm' : 'pondmen'}, axis = 1, inplace= True)
erfs_2017['pondindiv'] = erfs_2017['nbind'] * erfs_2017['pondmen']

erfs_2017['niveau_de_vie'] = erfs_2017['revdispm'] / erfs_2017['nb_uci']
erfs_2017['decile_indiv_niveau_vie'] = weighted_quantiles(erfs_2017['niveau_de_vie'], labels = np.arange(1,11), weights = erfs_2017['pondindiv'], return_quantiles=False)
erfs_2017_by_decile = df_weighted_average_grouped(erfs_2017, groupe = 'decile_indiv_niveau_vie', varlist = ['revdispm','niveau_de_vie'])

In [34]:
(erfs_2017['revdispm'] * erfs_2017['pondmen']).sum() * 1E-9

1046.961731829182

In [36]:
cn_aggregates = new_get_cn_aggregates(2017)
cn_aggregates.loc[cn_aggregates.index == 'rev_disponible','conso_CN_2017'] * 1e-9

Code
rev_disponible    1198.2253
Name: conso_CN_2017, dtype: float64

In [44]:
bdf_brut_2017_by_decile

,depenses_tot,rev_disponible,niveau_de_vie,part_epargnants,aise_1_2_3,taux_epargne
decile_indiv_niveau_vie,,,,,,
1.0,16624.379693,12033.365675,7771.597011,0.364042,0.556694,-0.381524
2.0,18672.769979,18695.775843,12194.837137,0.606430,0.646267,0.001231
3.0,20159.614604,22062.551965,14711.467432,0.666473,0.717900,0.086252
4.0,22786.992379,25747.384567,17021.682150,0.696474,0.800655,0.114978
5.0,24192.691589,28938.312244,19152.627451,0.723806,0.840285,0.163991
6.0,27054.696501,32627.493487,21407.492165,0.748634,0.875082,0.170801
7.0,29309.233033,37174.906606,23894.011292,0.782197,0.917643,0.211586
8.0,32712.273305,42383.481753,27136.563509,0.791964,0.927321,0.228183
9.0,35989.341316,50143.268038,32092.461224,0.855351,0.948395,0.282270


In [38]:
erfs_2017_by_decile

,revdispm,niveau_de_vie
decile_indiv_niveau_vie,,
1.0,10549.366141,6885.910338
2.0,19171.650407,12553.453549
3.0,22669.085425,15141.987546
4.0,26202.206028,17419.079931
5.0,29605.349803,19592.440636
6.0,33656.452165,21907.309100
7.0,38045.423074,24567.160571
8.0,43616.957178,27997.120623
9.0,51999.964338,33609.156754


In [39]:
bdf_cale_2017_by_decile

,depenses_tot,rev_disponible,niveau_de_vie,part_epargnants,aise_1_2_3,taux_epargne
decile_indiv_niveau_vie,,,,,,
1.0,20883.408955,14621.500769,9443.111331,0.378410,0.556694,-0.428267
2.0,23188.843481,22716.861486,14817.701543,0.633176,0.646267,-0.020777
3.0,25013.418878,26807.763471,17875.608407,0.674252,0.717900,0.066934
4.0,28424.274452,31285.129565,20682.703918,0.698420,0.800655,0.091445
5.0,31097.516979,35162.361638,23271.972713,0.721730,0.840285,0.115602
6.0,35293.039565,39645.011599,26011.813482,0.744380,0.875082,0.109774
7.0,36002.262567,45170.481892,29033.132894,0.775009,0.917643,0.202969
8.0,41812.952536,51499.316872,32973.092990,0.774569,0.927321,0.188087
9.0,44315.099879,60928.077225,38994.904632,0.842329,0.948395,0.272665


In [63]:
input_bdf_2017 = get_input_data_frame(2017)
input_bdf_2017_grouped = input_bdf_2017.filter(like='poste_').groupby(lambda col: '_'.join(col.split('_')[:3]), axis=1).sum()

to_drop = [col for col in input_bdf_2017_grouped.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
input_bdf_2017_grouped.drop(columns= to_drop, axis = 1, inplace= True)
input_bdf_2017_grouped = pd.concat([input_bdf_2017[['pondmen','npers','ident_men','rev_disponible','ocde10','aise','stalog','typmen']], input_bdf_2017_grouped], axis=1)

## Méthode 1 : redressement des revenus sur la base d’un filtre de cohérence revenu/consommation

Tiré de Bellamy et al. (2009): "Le redressement du revenu basé sur le filtre de cohérence revenu - consommation est le
suivant :
    (i) sur l’ensemble du fichier de l’enquête les très faibles revenus sont redressés : le
niveau de vie d’un ménage déclarant moins de 300 € par mois et par UC est
ramené automatiquement à 300 € par mois et par UC. Ce cas concerne 16
ménages sur les 10 240 de l’échantillon.
    (ii) On définit ensuite une "consommation courante" en éliminant les
consommations exceptionnelles.
    (iii) Si cette consommation courante est supérieure de plus de 20 % au revenu
courant (revenu constaté une fois ce dernier éventuellement mis à 300 € par
mois et par UC en (i)), soit pour 15,3 % des ménages, on effectue le test
suivant, analogue à celui utilisé par Loisy (1999).

Si le ménage :
1 - déclare être « à l’aise » financièrement, que « ça va » ou que « c’est juste, mais il
faut faire attention »
et
2 - déclare mettre de l’argent de côté ou que revenus et dépenses s’équilibrent
alors on remonte le revenu calculé en (i) au niveau de la consommation courante (ii). Dans
les autres cas, on laisse le revenu (i) et la consommation inchangés. Au total 10,7 % des
ménages sont affectés par ce redressement. On notera que ce traitement touche a priori
tous les ménages, quel que soit leur niveau de vie initial."

In [64]:
input_bdf_2017_grouped.drop('poste_04_2', axis= 1, inplace= True)        # on retire les loyers imputés des dépenses
input_bdf_2017_grouped = input_bdf_2017_grouped.apply(pd.to_numeric, errors='coerce')

liste_poste = [col for col in input_bdf_2017_grouped.columns.tolist() if col.startswith('poste')]
for poste in liste_poste:
   results = weighted_quantiles(input_bdf_2017_grouped['{}'.format(poste)], labels = np.arange(1,21), weights = input_bdf_2017_grouped['pondmen'], return_quantiles=True)
   input_bdf_2017_grouped['conso_courante_{}'.format(poste)] = input_bdf_2017_grouped['{}'.format(poste)].clip(upper=results[1][19])

input_bdf_2017_grouped['depenses_tot'] = input_bdf_2017_grouped[liste_poste].sum(axis = 1) 
input_bdf_2017_grouped['conso_courante_totale'] = input_bdf_2017_grouped.filter(like = 'conso_courante').sum(axis = 1)

input_bdf_2017_grouped['rev_disponible_2'] = input_bdf_2017_grouped['rev_disponible'].clip(lower= 3600 * input_bdf_2017_grouped['ocde10'])   

In [65]:
len(input_bdf_2017_grouped.loc[input_bdf_2017_grouped['rev_disponible_2'] > input_bdf_2017_grouped['rev_disponible']])

137

Il y a 137 ménages dont le niveau de vie est inférieur à 300€ par mois par UC.

In [66]:
input_bdf_2017_grouped['TEST'] = input_bdf_2017_grouped['conso_courante_totale'] > 1.2 * input_bdf_2017_grouped['rev_disponible_2']
input_bdf_2017_grouped['rev_disponible_3'] = input_bdf_2017_grouped['TEST']*(input_bdf_2017_grouped['aise'] <= 3)*input_bdf_2017_grouped['conso_courante_totale'] + input_bdf_2017_grouped['TEST'] * (input_bdf_2017_grouped['aise']>3) * input_bdf_2017_grouped['rev_disponible_2'] + (1 - input_bdf_2017_grouped['TEST']) * input_bdf_2017_grouped['rev_disponible_2']

In [67]:
input_bdf_2017_grouped['TEST'].sum(axis = 0)

1232

Il y a 1232 (825 quand le seuil est à p90) ménages dont la consommation courante est supérieure de plus de 20% à leur revenu disponible

In [68]:
(input_bdf_2017_grouped['TEST']*(input_bdf_2017_grouped['aise'] <= 3)).sum()

890

Dont 890 (559 quand le seuil est à p90) qui déclarent être « à l’aise » financièrement, que « ça va » ou que « c’est juste, mais il
faut faire attention ».

In [69]:
input_bdf_2017_grouped['part_epargnants'] = input_bdf_2017_grouped['rev_disponible_3'] >= input_bdf_2017_grouped['depenses_tot'] 
input_bdf_2017_grouped['part_epargnants_initiale'] = input_bdf_2017_grouped['rev_disponible'] >= input_bdf_2017_grouped['depenses_tot']

In [71]:
input_bdf_2017_grouped['pondindiv'] = input_bdf_2017_grouped['npers'] * input_bdf_2017_grouped['pondmen']
for rev in ['','_2','_3']:
    input_bdf_2017_grouped['niveau_de_vie{}'.format(rev)] = input_bdf_2017_grouped['rev_disponible{}'.format(rev)] / input_bdf_2017_grouped['ocde10']
    
input_bdf_2017_grouped['decile_indiv_niveau_vie_3'] = weighted_quantiles(input_bdf_2017_grouped['niveau_de_vie_3'], labels = np.arange(1,11), weights = input_bdf_2017_grouped['pondindiv'], return_quantiles=False)
input_bdf_2017_by_decile = df_weighted_average_grouped(input_bdf_2017_grouped, groupe = 'decile_indiv_niveau_vie_3', varlist = ['niveau_de_vie_3','rev_disponible_3','depenses_tot','pondmen','part_epargnants','part_epargnants_initiale'])

input_bdf_2017_by_decile['taux_epargne'] = 1 - input_bdf_2017_by_decile['depenses_tot'] / input_bdf_2017_by_decile['rev_disponible_3'] 

In [72]:
input_bdf_2017_by_decile

,niveau_de_vie_3,rev_disponible_3,depenses_tot,pondmen,part_epargnants,part_epargnants_initiale,taux_epargne
decile_indiv_niveau_vie_3,,,,,,,
1.0,8870.047890,13870.594505,14208.776642,3140.234542,0.548106,0.504947,-0.024381
2.0,12824.824834,19546.282103,17052.213569,3015.639235,0.681657,0.652485,0.127598
3.0,15280.530966,23403.316928,20192.046268,2928.113398,0.671712,0.663997,0.137214
4.0,17508.092291,25978.991113,21591.939618,3071.092906,0.705966,0.694074,0.168869
5.0,19638.261170,29939.650331,24209.424145,2872.515766,0.729557,0.718682,0.191393
6.0,21814.442111,33220.946586,27426.392924,2937.162837,0.730572,0.719146,0.174425
7.0,24262.404514,37595.777170,30186.929500,2994.762479,0.745023,0.739163,0.197066
8.0,27478.556999,42572.954991,33187.333283,2937.728071,0.749734,0.746818,0.220460
9.0,32334.160539,49580.006540,37403.114166,2868.578056,0.794068,0.793443,0.245601


In [73]:
erfs_2017_by_decile

,revdispm,niveau_de_vie
decile_indiv_niveau_vie,,
1.0,10549.366141,6885.910338
2.0,19171.650407,12553.453549
3.0,22669.085425,15141.987546
4.0,26202.206028,17419.079931
5.0,29605.349803,19592.440636
6.0,33656.452165,21907.309100
7.0,38045.423074,24567.160571
8.0,43616.957178,27997.120623
9.0,51999.964338,33609.156754


## Méthode 2 : Imputation des données de consommation par strates

### Méthode 2.1 : En utilisant le revenu disponible de BdF (brut)

In [75]:
to_drop = [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
input_bdf_2017.drop(to_drop, axis = 1, inplace = True)
input_bdf_2017['niveau_de_vie'] = input_bdf_2017['rev_disponible'] / input_bdf_2017['ocde10']
input_bdf_2017['niveau_de_vie'] = input_bdf_2017['niveau_de_vie'].astype(float)
input_bdf_2017['pondmen'] = input_bdf_2017['pondmen'].astype(float)
input_bdf_2017['npers'] = input_bdf_2017['npers'].astype(float)

input_bdf_2017['pondindiv'] = input_bdf_2017['npers'] * input_bdf_2017['pondmen'] 

input_bdf_2017['decile_indiv_niveau_vie'] = weighted_quantiles(data = input_bdf_2017['niveau_de_vie'], labels = np.arange(1,11), weights = input_bdf_2017['pondindiv'], return_quantiles=False)
input_bdf_2017['stalog_2'] = input_bdf_2017['stalog'].replace({1:1, 3:2, 4:2, 5:2})

In [77]:
def assign_strate_ines(row):
    if row['typmen'] == 5:
        return 'T5'
    elif row['typmen'] == 2:
        return f'T2_D{int(row["decile_indiv_niveau_vie"])}'
    elif row['typmen'] in [1, 3, 4]:
        return f'T{int(row["typmen"])}_D{int(row["decile_indiv_niveau_vie"])}_S{int(row["stalog_2"])}'
    else:
        return 'Other'

input_bdf_2017['strate_ines'] = input_bdf_2017.apply(assign_strate_ines, axis=1)

In [78]:
input_bdf_2017['strate_ines'].nunique()

71

In [80]:
input_bdf_2017[['decile_indiv_niveau_vie','typmen','stalog','strate_ines']]

,decile_indiv_niveau_vie,typmen,stalog,strate_ines
0,7.0,3,1,T3_D7_S1
1,10.0,4,2,T4_D10_S2
2,7.0,4,1,T4_D7_S1
3,2.0,1,3,T1_D2_S2
4,7.0,4,1,T4_D7_S1
...,...,...,...,...
12076,5.0,1,3,T1_D5_S2
12077,10.0,4,2,T4_D10_S2
12078,4.0,4,1,T4_D4_S1
12079,8.0,3,1,T3_D8_S1


In [81]:
input_bdf_2017['strate_ines'].value_counts()

T3_D10_S1    413
T2_D1        389
T2_D2        318
T1_D1_S2     309
T2_D3        306
            ... 
T4_D3_S1      54
T4_D2_S1      54
T3_D1_S1      51
T2_D10        48
T4_D1_S1      43
Name: strate_ines, Length: 71, dtype: int64

In [82]:
liste_var = ['rev_disponible'] + [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')]

input_bdf_2017_by_strate = df_weighted_average_grouped(input_bdf_2017, groupe = 'strate_ines', varlist = liste_var)
for poste in [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')] : 
    input_bdf_2017_by_strate['part_{}'.format(poste)] = input_bdf_2017_by_strate['{}'.format(poste)] / input_bdf_2017_by_strate['rev_disponible']
    
part_conso_by_strat = input_bdf_2017_by_strate.filter(like = 'part').reset_index()

C:\Users\veve1\AppData\Local\Temp/ipykernel_25368/897928326.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  input_bdf_2017_by_strate['part_{}'.format(poste)] = input_bdf_2017_by_strate['{}'.format(poste)] / input_bdf_2017_by_strate['rev_disponible']


In [83]:
new_input_bdf_2017 = input_bdf_2017.merge(part_conso_by_strat, how = 'left', left_on = 'strate_ines', right_on = 'strate_ines')
for poste in [col for col in input_bdf_2017.columns.tolist() if col.startswith('poste_')] : 
    new_input_bdf_2017['new_{}'.format(poste)] = new_input_bdf_2017['part_{}'.format(poste)] * new_input_bdf_2017['rev_disponible']
    
liste_new_poste = [col for col in new_input_bdf_2017.columns.tolist() if col.startswith('new_poste')]
new_input_bdf_2017['depenses_tot'] = new_input_bdf_2017[liste_new_poste].sum(axis = 1)

new_input_bdf_2017_by_decile = df_weighted_average_grouped(dataframe = new_input_bdf_2017, groupe = 'decile_indiv_niveau_vie', varlist =['depenses_tot','rev_disponible'])
new_input_bdf_2017_by_decile['taux_epargne'] = 1 - new_input_bdf_2017_by_decile['depenses_tot'] / new_input_bdf_2017_by_decile['rev_disponible']

C:\Users\veve1\AppData\Local\Temp/ipykernel_25368/720009144.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_input_bdf_2017['new_{}'.format(poste)] = new_input_bdf_2017['part_{}'.format(poste)] * new_input_bdf_2017['rev_disponible']


In [84]:
new_input_bdf_2017_by_decile

,depenses_tot,rev_disponible,taux_epargne
decile_indiv_niveau_vie,,,
1.0,18612.281500,11599.193347,-0.604619
2.0,21053.291203,18695.940417,-0.126089
3.0,23311.658745,22062.754790,-0.056607
4.0,26584.543601,25747.670334,-0.032503
5.0,28295.546328,28938.607224,0.022222
6.0,31657.044128,32627.717343,0.029750
7.0,34597.586832,37175.105100,0.069335
8.0,38757.980690,42383.685422,0.085545
9.0,43033.300463,50143.457903,0.141796


In [85]:
def imputation_depenses_ines(input_df):
    to_drop = [col for col in input_df.columns.tolist() if col.startswith('poste_17') or col.startswith('poste_18')]
    input_df.drop(to_drop, axis = 1, inplace = True)
    
    input_df['niveau_de_vie'] = input_df['rev_disponible'] / input_df['ocde10']
    input_df['niveau_de_vie'] = input_df['niveau_de_vie'].astype(float)
    input_df['pondmen'] = input_df['pondmen'].astype(float)
    input_df['decile_indiv_niveau_vie'] = weighted_quantiles(data = input_df['niveau_de_vie'], labels = np.arange(1,11), weights = input_df['pondmen'], return_quantiles=False)
    input_df['stalog_2'] = input_df['stalog'].replace({1:1, 3:2, 4:2, 5:2})
    
    input_df['strate_ines'] = input_df.apply(assign_strate_ines, axis=1)
    liste_var = ['rev_disponible'] + [col for col in input_df.columns.tolist() if col.startswith('poste_')]

    input_df_by_strate = df_weighted_average_grouped(input_df, groupe = 'strate_ines', varlist = liste_var)
    for poste in [col for col in input_df.columns.tolist() if col.startswith('poste_')] : 
        input_df_by_strate['part_{}'.format(poste)] = input_df_by_strate['{}'.format(poste)] / input_df_by_strate['rev_disponible']
    part_conso_by_strat = input_df_by_strate.filter(like = 'part').reset_index()
    
    new_input_df = input_df.merge(part_conso_by_strat, how = 'left', left_on = 'strate_ines', right_on = 'strate_ines')
    for poste in [col for col in input_df.columns.tolist() if col.startswith('poste_')] : 
        new_input_df['new_{}'.format(poste)] = new_input_df['part_{}'.format(poste)] * new_input_df['rev_disponible']
        
    liste_new_poste = [col for col in new_input_df.columns.tolist() if col.startswith('new_poste')]
    new_input_df['depenses_tot'] = new_input_df[liste_new_poste].sum(axis = 1)

    new_input_df_by_decile = df_weighted_average_grouped(dataframe = new_input_df, groupe = 'decile_indiv_niveau_vie', varlist =['depenses_tot','rev_disponible'])
    new_input_df_by_decile['taux_epargne'] = 1 - new_input_df_by_decile['depenses_tot'] / new_input_df_by_decile['rev_disponible']
    
    return(new_input_df,new_input_df_by_decile)

### Méthode 2.2 : En utilisant les consos et revenus BdF calés en 2017

In [86]:
input_bdf_cale_2017 = survey_scenario.create_data_frame_by_entity(liste_var + ['ocde10','pondmen','stalog','typmen'], period = 2017)['menage']

In [87]:
new_input_bdf_cale_2017, new_input_bdf_cale_2017_by_decile = imputation_depenses_ines(input_bdf_cale_2017)

C:\Users\veve1\AppData\Local\Temp/ipykernel_25368/3971310920.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df_by_strate['part_{}'.format(poste)] = input_df_by_strate['{}'.format(poste)] / input_df_by_strate['rev_disponible']
C:\Users\veve1\AppData\Local\Temp/ipykernel_25368/3971310920.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_input_df['new_{}'.format(poste)] = new_input_df['part_{}'.format(poste)] * new_input_df['rev_disponible']


In [88]:
new_input_bdf_cale_2017_by_decile

,depenses_tot,rev_disponible,taux_epargne
decile_indiv_niveau_vie,,,
1.0,16093.931903,11578.863640,-0.389941
2.0,18592.600141,18776.227846,0.009780
3.0,20118.849261,22042.378099,0.087265
4.0,22756.084526,25739.881780,0.115921
5.0,23998.455710,28714.357431,0.164235
6.0,26919.282015,32428.782916,0.169895
7.0,29319.183157,37092.999439,0.209576
8.0,32752.530780,42136.181623,0.222698
9.0,36111.004428,50048.298501,0.278477


In [89]:
new_input_bdf_cale_2017_by_decile[['depenses_tot','rev_disponible']].sum(axis = 0)

depenses_tot      275495.034475
rev_disponible    345077.088554
dtype: float64

In [90]:
1 - 275495/ 345077

0.2016419523758466

In [91]:
new_input_bdf_2017_by_decile

,depenses_tot,rev_disponible,taux_epargne
decile_indiv_niveau_vie,,,
1.0,18612.281500,11599.193347,-0.604619
2.0,21053.291203,18695.940417,-0.126089
3.0,23311.658745,22062.754790,-0.056607
4.0,26584.543601,25747.670334,-0.032503
5.0,28295.546328,28938.607224,0.022222
6.0,31657.044128,32627.717343,0.029750
7.0,34597.586832,37175.105100,0.069335
8.0,38757.980690,42383.685422,0.085545
9.0,43033.300463,50143.457903,0.141796


### Méthode 2.3 : En utilisant les revenus calés sur ceux de TaxIPP et les dépenses calées pour 13 Mds€

In [27]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import new_get_cn_aggregates

In [29]:
cn_agregates = new_get_cn_aggregates(2017)

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [33]:
cn_agregates.loc[cn_agregates.index == 'poste_06']

,conso_CN_2017
Code,
